In [1]:
import os 
%pwd

'd:\\My Workspace\\proj_file\\End-To-End-Wine-Quality-Pred\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\My Workspace\\proj_file\\End-To-End-Wine-Quality-Pred'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DatatransformationConfig:
    root_dir: Path
    data_path: Path


In [4]:
from src.constants import *
from src.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_data_transformation_config(self) -> DatatransformationConfig:
        
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        
        data_transformation_config = DatatransformationConfig(
            root_dir= config.root_dir,
            data_path=config.data_path,
        )
        
        return data_transformation_config

In [7]:
import os
import logging
from sklearn.model_selection import train_test_split
import pandas as pd

class DataTransformation:
    def __init__(self, config: DatatransformationConfig):
        self.config = config

    def preprocessing(self):
        df = pd.read_csv(self.config.data_path)
        
        df['quality']=df['quality'].apply(lambda y_value:1 if y_value>=7 else 0)
        
        tar_col = df['quality']
        
        train,test = train_test_split(df,random_state=2,test_size=0.2,stratify=tar_col)
        
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logging.info("Splited data into training and test sets")
        logging.info(train.shape)
        logging.info(test.shape)
        logging.info(train.head())
        logging.info(test.head())

        print(train.shape)
        print(test.shape)
        print(train.head())
        print(test.head())

In [8]:
import sys 
from src.exception import CustomException
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.preprocessing()
except Exception as e:
    raise CustomException(e,sys)


[ 2024-06-22 18:24:48,995 ] 33 root - INFO - yaml file: config\config.yaml loaded successfully
[ 2024-06-22 18:24:48,997 ] 33 root - INFO - yaml file: params.yaml loaded successfully
[ 2024-06-22 18:24:49,000 ] 33 root - INFO - yaml file: schema.yaml loaded successfully
[ 2024-06-22 18:24:49,001 ] 53 root - INFO - created directory at: artifacts
[ 2024-06-22 18:24:49,002 ] 53 root - INFO - created directory at: artifacts/data_transformation
[ 2024-06-22 18:24:49,027 ] 22 root - INFO - Splited data into training and test sets
[ 2024-06-22 18:24:49,028 ] 23 root - INFO - (1279, 12)
[ 2024-06-22 18:24:49,029 ] 24 root - INFO - (320, 12)
[ 2024-06-22 18:24:49,030 ] 25 root - INFO -       fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
1031            7.3             0.550         0.01             1.8      0.093   
1364            7.2             0.605         0.02             1.9      0.096   
649             6.7             0.420         0.27             8.6    